In [40]:
from vit_3d import ViT

import torch 

image_encoder = ViT(
            image_size = 256,          # image size
            frames = 512,               # max number of frames
            image_patch_size = 32,     # image patch size
            frame_patch_size = 4,      # frame patch size
            dim = 768,
            depth = 12,
            heads = 8,
            mlp_dim = 2048,
            channels=1,
            dropout = 0.1,
            emb_dropout = 0.1
        )

video = torch.randn(1, 1, 256, 256, 156)

video_encoding, pos = image_encoder(video) # (1, 512, 768)




: 

In [41]:
# video = torch.randn(8, 1, 512, 512, 8)

# video_encoding, pos = image_encoder(video) 
video_encoding.shape

torch.Size([8, 64, 768])

In [42]:
from einops import rearrange, repeat
T = video_encoding.shape[1]
# ([8, 512, 768]) --> [8, 2, 256, 768]
video_encoding = rearrange(video_encoding, 'b (t f) d -> b t f d', f=64)


In [43]:
video_encoding.shape, 64 * 768

(torch.Size([8, 1, 64, 768]), 49152)

In [19]:
from ctvit import CTViT

image_encoder1 = CTViT(
    dim = 512,
    codebook_size = 8192,
    image_size = 480,
    patch_size = 20,
    temporal_patch_size = 10,
    spatial_depth = 4,
    temporal_depth = 4,
    dim_head = 32,
    heads = 8
).to('cuda')
video = torch.randn(8, 1, 30, 480, 480).to('cuda')
tokens = image_encoder1(video, return_encoded_tokens=True)


/home/anaconda3/envs/BBDM/lib/python3.9/site-packages/torch/nn/modules/conv.py:608: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv3d(


In [38]:
video = torch.randn(8, 1, 40, 480, 480).to('cuda')
tokens = image_encoder1(video, return_encoded_tokens=True)

In [39]:
tokens.shape

torch.Size([8, 4, 24, 24, 512])

In [21]:
24 * 24 * 512

294912

In [1]:

root_dir = "./DAC001"  # Replace with your actual root folder path

# Create datasets for different splits.
train_dataset = MedicalImageReportDataset(root=root_dir, split='train')
val_dataset   = MedicalImageReportDataset(root=root_dir, split='val')
test_dataset  = MedicalImageReportDataset(root=root_dir, split='test')

print("Number of training samples:", len(train_dataset))
print("Number of validation samples:", len(val_dataset))
print("Number of test samples:", len(test_dataset))

# Display a sample
image, report = train_dataset[0]
print("Image shape:", image.shape)
print("Report content:", report)

FileNotFoundError: [Errno 2] No such file or directory: './DAC001'

In [1]:
!pip install monai

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 50.7 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.1
    Uninstalling numpy-2.2.1:
      Successfully uninstalled numpy-2.2.1


In [1]:
import os
import sys
import numpy as np
from monai.data import CacheDataset
import torch
from monai.transforms import LoadImaged
from torch.utils.data import DataLoader

class PetReportPairDataset(CacheDataset):
    def __init__(self, 
                 root, 
                 split='train', 
                 transform=LoadImaged("image"), 
                 cache_num=sys.maxsize,
                 cache_rate=1.0,
                 num_workers=4,):
        """
        Args:
            root (str): Path to the root folder (e.g., "./DAC001").
            split (str): One of 'train', 'val', or 'test'.
                - train: use all month folders except THANG 10, THANG 11, THANG 12.
                - val: use only THANG 10.
                - test: use only THANG 11 and THANG 12.
            transform: Optional transform to be applied on a sample (e.g., conversion to torch tensor, normalization, etc.).
        """
        self.root = root
        self.split = split.lower()
        self.transform = transform
        
        # Determine which month folders to include based on the split.
        self.month_folders = []
        for month in os.listdir(root):
            month_path = os.path.join(root, month)
            if not os.path.isdir(month_path):
                continue
            if self.split == 'train':
                if month in ['THANG 10', 'THANG 11', 'THANG 12']:
                    continue
                else:
                    self.month_folders.append(month_path)
            elif self.split == 'val':
                if month == 'THANG 10':
                    self.month_folders.append(month_path)
            elif self.split == 'test':
                if month in ['THANG 11', 'THANG 12']:
                    self.month_folders.append(month_path)
        
        # Allowed modalities (exclude "whole_body")
        allowed_modalities = ['abdomen_pelvis', 'chest', 'head_neck']
        
        # Build the list of (image_path, report_path) pairs.
        self.datalist = []
        for month_folder in self.month_folders:
            images_root = os.path.join(month_folder, 'images')
            reports_root = os.path.join(month_folder, 'reports')
            if not os.path.isdir(images_root) or not os.path.isdir(reports_root):
                continue
            for modality in allowed_modalities:
                modality_img_folder = os.path.join(images_root, modality)
                modality_rep_folder = os.path.join(reports_root, modality)
                if not os.path.isdir(modality_img_folder) or not os.path.isdir(modality_rep_folder):
                    continue
                # List all image files ending with .npy
                image_files = sorted([f for f in os.listdir(modality_img_folder) if f.endswith('.npy')])
                for img_file in image_files:
                    base_name = os.path.splitext(img_file)[0]
                    rep_file = base_name + '.txt'
                    img_file_path = os.path.join(modality_img_folder, img_file)
                    rep_file_path = os.path.join(modality_rep_folder, rep_file)
                    if os.path.exists(rep_file_path):
                        self.datalist.append({"image": img_file_path, "label": rep_file_path})
        
        super().__init__(
            data=self.datalist,
            transform=self.transform,
            cache_num=cache_num,
            cache_rate=cache_rate,
            num_workers=num_workers,
        )

    def __getitem__(self, index):
        # Load sample
        sample = self.datalist[index]
        image_path = sample['image']
        report_path = sample['label']

        # Load the image data
        image = np.load(image_path)
        
        # Padding depth to be divisible by 4
        depth = image.shape[0]
        if depth % 4 != 0:
            padding_needed = 4 - (depth % 4)
            image = np.pad(image, ((0, padding_needed), (0, 0), (0, 0)), mode='constant', constant_values=0)

        # Normalize image to range [-1, 1]
        image = image.astype(np.float32)  # Ensure float32 for the normalization
        image = (image - 400) / 600  # Normalize to [0, 1]
        image = 2 * image - 1  # Scale to [-1, 1]

        # Apply the transform (e.g., to tensor, resize, etc.)
        if self.transform:
            image = self.transform({"image": image})['image']

        # Load the report text
        with open(report_path, 'r', encoding='utf-8') as file:
            report_text = file.read().strip()

        return image, report_text

# Example usage:
# if __name__ == '__main__':
data_folder = "/home/user01/aiotlab/thaind/DAC001"  # Path to your root folder
dataset = PetReportPairDataset(root=data_folder, split='train', transform=LoadImaged("image"))
print(f"Dataset length: {len(dataset)}")

# Fetch and display one sample
image, report = dataset[0]
print("Image shape:", image.shape)
print("Report text:", report)

/home/user01/miniconda3/envs/thaind/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading dataset:  25%|██▍       | 743/2988 [02:17<16:02,  2.33it/s]  

: 

In [3]:
import os
import numpy as np
import pandas as pd

# Định nghĩa đường dẫn tới dataset
DATASET_PATH = "/home/user01/aiotlab/thaind/DAC001"

# Các bộ phận cần thống kê
BODY_PARTS = ["abdomen_pelvis", "chest", "head_neck"]

# Dictionary để lưu thống kê độ sâu
depth_stats = {part: [] for part in BODY_PARTS}
depth_stats["all"] = []  # Thống kê toàn bộ dataset

# Duyệt qua từng tháng trong dataset
for month in os.listdir(DATASET_PATH):
    month_path = os.path.join(DATASET_PATH, month)
    
    if not os.path.isdir(month_path):
        continue  # Bỏ qua nếu không phải thư mục
    
    images_path = os.path.join(month_path, "images")
    
    if not os.path.isdir(images_path):
        continue  # Bỏ qua nếu không có thư mục hình ảnh
    
    # Duyệt qua từng bộ phận
    for part in BODY_PARTS:
        part_path = os.path.join(images_path, part)
        
        if not os.path.isdir(part_path):
            continue  # Bỏ qua nếu thư mục không tồn tại
        
        # Duyệt qua tất cả các file .npy
        for file in os.listdir(part_path):
            if file.endswith(".npy"):
                file_path = os.path.join(part_path, file)
                
                # Load file npy và lấy độ sâu
                try:
                    image = np.load(file_path)
                    depth = image.shape[0]  # Lấy kích thước chiều sâu
                    depth_stats[part].append(depth)
                    depth_stats["all"].append(depth)  # Lưu vào thống kê toàn bộ dataset
                except Exception as e:
                    print(f"Lỗi khi đọc {file_path}: {e}")

# Hàm tính toán thống kê cơ bản
def compute_statistics(depth_list):
    if not depth_list:
        return {"min": None, "max": None, "mean": None, "std": None, "count": 0}
    
    return {
        "min": np.min(depth_list),
        "max": np.max(depth_list),
        "mean": np.mean(depth_list),
        "std": np.std(depth_list),
        "count": len(depth_list)
    }

# Tạo DataFrame để hiển thị kết quả
stats_summary = {part: compute_statistics(depth_stats[part]) for part in depth_stats}
df_stats = pd.DataFrame(stats_summary).T

# Hiển thị thống kê
# import ace_tools as tools  # Sử dụng công cụ hiển thị DataFrame
# tools.display_dataframe_to_user(name="Dataset Depth Statistics", dataframe=df_stats)

In [14]:
df_stats

,min,max,mean,std,count
abdomen_pelvis,69.0,268.0,155.817696,15.757686,1311.0
chest,1.0,218.0,142.905199,10.534502,1308.0
head_neck,50.0,161.0,98.756674,8.605844,1311.0
all,1.0,268.0,132.485242,27.234360,3930.0


In [16]:
import os
import numpy as np

# Định nghĩa đường dẫn tới dataset
DATASET_PATH = "/home/user01/aiotlab/thaind/DAC001"

# Bộ phận cần lọc
TARGET_PART = "whole_body"

# Danh sách lưu các tệp có hơn 100 slices
large_slices_files = []

# Duyệt qua từng tháng trong dataset
for month in os.listdir(DATASET_PATH):
    month_path = os.path.join(DATASET_PATH, month)

    if not os.path.isdir(month_path):
        continue  # Bỏ qua nếu không phải thư mục
    
    images_path = os.path.join(month_path, "images")
    part_path = os.path.join(images_path, TARGET_PART)

    if not os.path.isdir(part_path):
        continue  # Bỏ qua nếu thư mục không tồn tại
    
    # Duyệt qua tất cả các file .npy trong thư mục head_neck
    for file in os.listdir(part_path):
        if file.endswith(".npy"):
            file_path = os.path.join(part_path, file)

            # Load file npy và lấy số lát cắt (depth)
            try:
                image = np.load(file_path)
                depth = image.shape[0]  # Lấy kích thước chiều sâu (số lát cắt)

                if depth < 200:
                    large_slices_files.append((file_path, depth))  # Lưu lại đường dẫn và depth

            except Exception as e:
                print(f"Lỗi khi đọc {file_path}: {e}")

# In danh sách các file có hơn 100 lát cắt
if large_slices_files:
    print("\n🔍 Các file head_neck có hơn 110 lát cắt:")
    for path, depth in large_slices_files:
        print(f"{path} - Slices: {depth}")
else:
    print("✅ Không có file head_neck nào có hơn 100 lát cắt.")


🔍 Các file head_neck có hơn 110 lát cắt:
/home/user01/aiotlab/thaind/DAC001/THANG 3/images/whole_body/day_8_patient_317.npy - Slices: 199
/home/user01/aiotlab/thaind/DAC001/THANG 2/images/whole_body/day_24_patient_112.npy - Slices: 191
/home/user01/aiotlab/thaind/DAC001/THANG 12/images/whole_body/day_19_patient_1609.npy - Slices: 173
/home/user01/aiotlab/thaind/DAC001/THANG 4/images/whole_body/day_27_patient_458.npy - Slices: 89
/home/user01/aiotlab/thaind/DAC001/THANG 4/images/whole_body/day_25_patient_432.npy - Slices: 89
/home/user01/aiotlab/thaind/DAC001/THANG 7/images/whole_body/day_6_patient_1012.npy - Slices: 89
/home/user01/aiotlab/thaind/DAC001/THANG 9/images/whole_body/day_28_patient_1191.npy - Slices: 83
/home/user01/aiotlab/thaind/DAC001/THANG 11/images/whole_body/day_21_patient_1472.npy - Slices: 199


In [20]:
import os

# Danh sách các phần cơ thể cần xóa
TARGET_PARTS = ["whole_body", "head_neck", "chest", "abdomen_pelvis"]

# Danh sách file cần xóa (bạn có thể đọc từ file hoặc liệt kê trực tiếp)
files_to_delete = [
    "/home/user01/aiotlab/thaind/DAC001/THANG 3/images/abdomen_pelvis/day_8_patient_317.npy",
    "/home/user01/aiotlab/thaind/DAC001/THANG 2/images/abdomen_pelvis/day_24_patient_112.npy",
    "/home/user01/aiotlab/thaind/DAC001/THANG 12/images/abdomen_pelvis/day_19_patient_1609.npy",
    "/home/user01/aiotlab/thaind/DAC001/THANG 4/images/abdomen_pelvis/day_27_patient_458.npy",
    "/home/user01/aiotlab/thaind/DAC001/THANG 4/images/abdomen_pelvis/day_25_patient_432.npy",
    "/home/user01/aiotlab/thaind/DAC001/THANG 7/images/abdomen_pelvis/day_6_patient_1012.npy",
    "/home/user01/aiotlab/thaind/DAC001/THANG 9/images/abdomen_pelvis/day_28_patient_1191.npy",
    "/home/user01/aiotlab/thaind/DAC001/THANG 11/images/abdomen_pelvis/day_21_patient_1472.npy",
]

# Kiểm tra và xóa các file thuộc TARGET_PARTS
deleted_files = []
not_found_files = []

for file_path in files_to_delete:
    # Kiểm tra xem file có thuộc TARGET_PARTS không
    if any(part in file_path for part in TARGET_PARTS):
        if os.path.exists(file_path):
            try:
                os.remove(file_path)  # Xóa file
                deleted_files.append(file_path)
            except Exception as e:
                print(f"❌ Lỗi khi xóa {file_path}: {e}")
        else:
            not_found_files.append(file_path)

# Hiển thị kết quả
if deleted_files:
    print("\n🗑️ Các file đã xóa:")
    for file in deleted_files:
        print(file)
else:
    print("✅ Không có file nào để xóa.")

if not_found_files:
    print("\n⚠️ Các file không tìm thấy (có thể đã bị xóa trước đó):")
    for file in not_found_files:
        print(file)


🗑️ Các file đã xóa:
/home/user01/aiotlab/thaind/DAC001/THANG 3/images/abdomen_pelvis/day_8_patient_317.npy
/home/user01/aiotlab/thaind/DAC001/THANG 2/images/abdomen_pelvis/day_24_patient_112.npy
/home/user01/aiotlab/thaind/DAC001/THANG 12/images/abdomen_pelvis/day_19_patient_1609.npy
/home/user01/aiotlab/thaind/DAC001/THANG 4/images/abdomen_pelvis/day_27_patient_458.npy
/home/user01/aiotlab/thaind/DAC001/THANG 4/images/abdomen_pelvis/day_25_patient_432.npy
/home/user01/aiotlab/thaind/DAC001/THANG 7/images/abdomen_pelvis/day_6_patient_1012.npy
/home/user01/aiotlab/thaind/DAC001/THANG 9/images/abdomen_pelvis/day_28_patient_1191.npy
/home/user01/aiotlab/thaind/DAC001/THANG 11/images/abdomen_pelvis/day_21_patient_1472.npy


In [10]:
# path = '/home/user01/aiotlab/thaind/DAC001/THANG 9/images/whole_body/day_28_patient_1191.npy'
path = '/home/user01/aiotlab/thaind/DAC001/THANG 12/images/whole_body/day_6_patient_1685.npy'
check = np.load(path)

check.shape

(515, 256, 256)

In [15]:
import os
import numpy as np

# Định nghĩa đường dẫn tới dataset
DATASET_PATH = "/home/user01/aiotlab/thaind/DAC001"

# Các bộ phận cần kiểm tra
BODY_PARTS = ["abdomen_pelvis", "chest", "head_neck"]

# Danh sách lưu các tệp có depth = 1
depth_1_files = []

# Duyệt qua từng tháng trong dataset
for month in os.listdir(DATASET_PATH):
    month_path = os.path.join(DATASET_PATH, month)

    if not os.path.isdir(month_path):
        continue  # Bỏ qua nếu không phải thư mục
    
    images_path = os.path.join(month_path, "images")

    if not os.path.isdir(images_path):
        continue  # Bỏ qua nếu không có thư mục hình ảnh
    
    # Duyệt qua từng bộ phận
    for part in BODY_PARTS:
        part_path = os.path.join(images_path, part)

        if not os.path.isdir(part_path):
            continue  # Bỏ qua nếu thư mục không tồn tại
        
        # Duyệt qua tất cả các file .npy
        for file in os.listdir(part_path):
            if file.endswith(".npy"):
                file_path = os.path.join(part_path, file)

                # Load file npy và lấy độ sâu
                try:
                    image = np.load(file_path)
                    depth = image.shape[0]  # Lấy kích thước chiều sâu

                    if depth == 1:
                        depth_1_files.append(file_path)  # Lưu lại đường dẫn

                except Exception as e:
                    print(f"Lỗi khi đọc {file_path}: {e}")

# In danh sách các file có Depth = 1
if depth_1_files:
    print("\n🔍 Các file có Depth = 1:")
    for path in depth_1_files:
        print(path)
else:
    print("✅ Không có file nào có Depth = 1.")


🔍 Các file có Depth = 1:
/home/user01/aiotlab/thaind/DAC001/THANG 9/images/chest/day_28_patient_1191.npy
